# Process results on instability prediction

In [ ]:
import os, sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy
from copy import deepcopy
from pickle import load
import re
from sklearn.preprocessing import StandardScaler
import random
from ast import literal_eval as make_tuple
from sklearn.pipeline import Pipeline
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split
from skorch import NeuralNetClassifier, NeuralNetRegressor
from combat.pycombat import pycombat

sys.path.insert(0, '../read_data/')
from read_data import read_data
from read_GDSC_response import read_GDSC_response
import library_size_normalization

from matplotlib import font_manager as fm, rcParams
fpath = os.path.join(rcParams["datapath"], "fonts/ttf/arial.ttf")
prop_label = fm.FontProperties(fname=fpath)
prop_label.set_size(30)
prop_ticks = fm.FontProperties(fname=fpath)
prop_ticks.set_size(25)
fname = os.path.split(fpath)[1]

sys.path.insert(0, '../src/')
from clf_utils import make_network, make_drug_folder
from cv_results_processing import parse_folder_results, read_best_param, make_skorch_network


sns.set_style("whitegrid")
sns.set_context('paper')

data_type = 'TCGA'
baseline = 'baseline_C'
test = 'Mann-Whitney-ls'

figure_folder = './figures/%s/'%(baseline)

## Neural network results

In [ ]:
relevant_folders = [f for f in os.listdir(results_folder) if os.path.isdir(results_folder + f)]

In [ ]:
rank_scores_df = {}

for drug_folder in relevant_folders:
    files = os.listdir(results_folder + drug_folder)
    unique_training_ids = [re.search('ID_([a-z0-9]*)_', f) for f in files]
    unique_training_ids = [e.group(0).replace('ID_', '').replace('_', '') 
                           for e in unique_training_ids if e is not None]
    unique_training_ids = np.unique(unique_training_ids).astype(str)
    print(len(unique_training_ids))
    
    # Load results
    global_rank_df = pd.DataFrame()
    all_rank_files = {training_id:[e for e in files 
                                    if (training_id in e) and (test in e) and ('ustat.txt' in e)][0]
                      for training_id in unique_training_ids}
    all_rank_scores = {
        training_id: pd.read_csv(results_folder + '/' + drug_folder + '/' + f, 
                                 header=None if data_type == 'TCGA' else 0,
                                 index_col=0)
        for training_id, f in all_rank_files.items()
    }
    all_rank_scores = {
        training_id: df[1] if data_type == 'TCGA' else df['PR-PD']
        for training_id, df in all_rank_scores.items()
    }
    rank_scores_df[drug_folder] = pd.DataFrame(all_rank_scores).T
    rank_scores_df[drug_folder]['AUC'] = rank_scores_df[drug_folder]['ustat'] / rank_scores_df[drug_folder]['product_samples']

### Load TRANSACT results

In [ ]:
transact_results_file = './figures/%s_ustat_summary.csv'%(data_type)
transact_df = pd.read_csv(transact_results_file).dropna()
transact_df['drug'] = 'GDSC_' +\
                    transact_df['GDSC_drug'] + \
                    '_%s_'%(data_type) + \
                    transact_df['%s_drug'%(data_type)].apply(lambda x: re.sub(r'[0-9]*$', '', x))

### Plot different target results on boxplots

In [ ]:
palette = 'Spectral'

if data_type == 'TCGA':
    fig = plt.figure(figsize=(12,8))
else:
    fig = plt.figure(figsize=(5,8))
    
drug_AUC_df = {f:df['AUC'].values for f, df in rank_scores_df.items()}
min_length = np.min([e.shape[0] for e in drug_AUC_df.values()])
drug_AUC_df = {f:df[:min_length] for f,df in drug_AUC_df.items()}
drug_AUC_df = pd.DataFrame(drug_AUC_df).melt()
AUC_order = transact_df.set_index('drug')['(\'TRANSACT\', \'AUC\')'].sort_values(ascending=True).index
ax = sns.swarmplot(x='variable',
                   y='value',
                   data=drug_AUC_df,
                   order=AUC_order,
                   alpha=0.5,
                   color='green',
                   label='Deep Learning')

sns.boxplot(data=drug_AUC_df, x='variable', y='value',
            order=AUC_order,
            color='green',
            linewidth=2.,
            width=.8,
            whis=[5,95],
            showfliers=False,
            boxprops=dict(alpha=.6))

plt.plot(transact_df.set_index('drug').loc[AUC_order]['(\'TRANSACT\', \'AUC\')'],
         linewidth=3,
         marker='P',
         markersize=10,
         label='TRANSACT')

plt.xticks(fontsize=15, rotation='vertical', color='black')
ax.xaxis.set_ticklabels([e.replace('GDSC_', 'GDSC:').replace('%s_'%(data_type), '%s:'%(data_type)).replace('_', '\n')
                         for e in AUC_order])
plt.xlabel(None)

plt.ylabel('Area under ROC (%s)'%(data_type), fontsize=25, color='black')
plt.yticks(fontsize=15, color='black')
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
if data_type == 'TCGA':
    plt.legend(by_label.values(), by_label.keys(), fontsize=20)
else:
    pass
plt.tight_layout()
plt.savefig('%s/DL_instability_results_%s_iter_%s.png'%(figure_folder, data_type, min_length),
            dpi=300)

## Correlation between results on GDSC and target
Does good results on GDSC imply good results on target ? Or is there a lot of local minima ?
### Load data

In [ ]:
# Data
tissues = {
    'GDSC': ['All'],
    data_type: [data_type if data_type=='TCGA' else 'all']
}
projects = {
    'GDSC': [None],
    data_type: ['all' if data_type=='TCGA' else None]
}

data_types = ['rnaseq']
data_sources = ['GDSC', data_type]
genes_filtering = 'mini'
data_normalization = 'library_size' # Can be TPM, "library_size" or "log". Else will not have any influence.
source = 'GDSC'
target = data_type

In [ ]:
if 'baseline_B' in figure_folder:
    data_df = read_data(tissues=tissues,
                        data_types=[e for e in data_types],
                        projects=projects,
                        data_sources=data_sources,
                        folder_basis='../data/')
    data_df = data_df['GDSC']

    # Harmonize and process
    while type(data_df) == dict:
        keys = data_df.keys()
        if len(keys) != 1:
            print('WARNING')
            break
        data_df = data_df[list(keys)[0]]

    GE_normalized = library_size_normalization.TMM_normalization(data_df.values.astype(float))
    GE_normalized = np.array(GE_normalized)
    GE_normalized = np.log(np.array(GE_normalized)+1)
    data_df = pd.DataFrame(GE_normalized,
                           columns=data_df.columns,
                           index=data_df.index)

    # Standardization and processing:
    scaler = StandardScaler(with_mean=True, with_std=True)
    data_df = pd.DataFrame(scaler.fit_transform(data_df),
                           index=data_df.index,
                           columns=data_df.columns)
    
elif 'baseline_C' in figure_folder:
    data_df = read_data(tissues=tissues,
                            data_types=[e for e in data_types],
                            projects=projects,
                            data_sources=data_sources,
                            folder_basis='../data/')
    
    for ds in list(data_df.keys()):
        assert len(data_df[ds].keys()) == 1
        new_key = ('%s_%s'%(ds, list(data_df[ds].keys())[0])).replace('fpkm', 'tpm')
        data_df[new_key] = data_df[ds][list(data_df[ds].keys())[0]]
        del data_df[ds]

    source_data_key = [ds for ds in data_df if source in ds]
    assert len(source_data_key) == 1
    source_data_key = np.unique(source_data_key)[0]

    target_data_key = [ds for ds in data_df if target in ds]
    assert len(target_data_key) == 1
    target_data_key = np.unique(target_data_key)[0]

    # Removing the healthy samples
    healthy_samples_index = data_df[target_data_key].index.str.contains(r'-(10A|11A)-')
    data_df[target_data_key] = data_df[target_data_key].loc[~healthy_samples_index]
    
    for ds in list(data_df.keys()):
        GE_normalized = library_size_normalization.TMM_normalization(data_df[ds].values.astype(float))
        GE_normalized = np.array(GE_normalized)
        GE_normalized = np.log(np.array(GE_normalized)+1)
        data_df[ds] = pd.DataFrame(GE_normalized,
                                   columns=data_df[ds].columns,
                                   index=data_df[ds].index)
    
    
    # remove some genes to avoid ComBat to collapse
    number_top_genes = 1700

    top_source_variable_genes = pd.DataFrame(np.var(data_df[source_data_key]), columns=['variance'])
    top_source_variable_genes = top_source_variable_genes.sort_values('variance', ascending=False)
    top_source_variable_genes = top_source_variable_genes.head(number_top_genes).index
    top_target_variable_genes = pd.DataFrame(np.var(data_df[target_data_key]), columns=['variance'])
    top_target_variable_genes = top_target_variable_genes.sort_values('variance', ascending=False)
    top_target_variable_genes = top_target_variable_genes.head(number_top_genes).index
    top_variable_genes = np.intersect1d(top_source_variable_genes, top_target_variable_genes)

    for d in data_df:
        data_df[d] = data_df[d][top_variable_genes]
        
    
    ## Correct with ComBat
    data_corrected = pycombat(pd.concat(list(data_df.values())).T,
                             [1]*data_df[source_data_key].shape[0] + [2]*data_df[target_data_key].shape[0])

    normalized_data_df = {
        k : data_corrected[data_df[k].index].T
        for k in data_df
    }
    normalized_data_df[source_data_key].index = pd.MultiIndex.from_tuples(normalized_data_df[source_data_key].index)
    data_df = normalized_data_df[source_data_key]

# # Response
unique_drugs = None
GDSC_drug_response_frames = {}
for x in ['GDSC2', 'GDSC1']:
    GDSC_drug_response_file = '../data/GDSC/response/%s_fitted_dose_response_25Feb20.xlsx'%(x)
    GDSC_drug_response_frames[x] = pd.read_excel(GDSC_drug_response_file)
    if unique_drugs is None:
        unique_drugs = np.unique(GDSC_drug_response_frames[x]['DRUG_NAME'])
    else:
        unique_drugs = np.concatenate([unique_drugs, np.unique(GDSC_drug_response_frames[x]['DRUG_NAME'])])

### Compare MSE on GDSC (source) to AUC on TCGA/HMF (target)

In [ ]:
if data_type == 'TCGA':
    drug_list =[
        ('Oxaliplatin', 1806, 'Oxaliplatin'),
        ('Cisplatin', None, 'Cisplatin'),
        ('Cisplatin', None, 'Carboplatin'),
        ('Afatinib', None, 'Trastuzumab'),
        ('Gemcitabine', None, 'Gemcitabine'),
        ('Paclitaxel', None, 'Paclitaxel'),
        ('Vinorelbine', None, 'Vinorelbine'),
        ('5-Fluorouracil', None, 'Fluorouracil'),
        ('Temozolomide', None, 'Temozolomide'),
        ('Doxorubicin', 133, 'Doxorubicin'),
        ('Docetaxel', 1819, 'Docetaxel'),
        ('Cyclophosphamide', None, 'Cyclophosphamide'),
        ('Etoposide', None, 'Etoposide'),
        ('Bleomycin', None, 'Bleomycin'),
        ('Pemetrexed', None, 'Pemetrexed'),
        ('Irinotecan', None, 'Irinotecan'),
        ('Cetuximab', None, 'Cetuximab'),
    ]
else:
    drug_list =[
        ('Cisplatin', None, 'Carboplatin'),
        ('Afatinib', None, 'Trastuzumab'),
        ('Gemcitabine', None, 'Gemcitabine'),
        ('Paclitaxel', None, 'Paclitaxel'),
        ('5-Fluorouracil', None, 'Fluorouracil'),
        ('Irinotecan', None, 'Irinotecan'),
    ]

In [ ]:
performance_GDSC_df = {}
predictions_all_drugs = {}

for GDSC_drug_name, GDSC_drug_ID, target_drug_name in drug_list:
    print(GDSC_drug_name, target_drug_name)
    X_source_response, y_source = read_GDSC_response(GDSC_drug_response_frames,
                                                     GDSC_drug_name,
                                                     data_df, GDSC_drug_ID)
    
    # Read drug gile
    drug_folder = [f for f in os.listdir(results_folder) 
                   if os.path.isdir(results_folder + f) 
                   and ('GDSC_%s'%(GDSC_drug_name) in f)
                   and ('%s_%s'%(data_type, target_drug_name) in f)]

    if len(drug_folder) != 1:
        print('WARNING: MORE THAN ONE FOLDER')
        print(drug_folder)
        del drug_folder
    else:
        drug_folder = drug_folder[0]
        
    clf_files = os.listdir(results_folder + drug_folder)
    clf_files = [f for f in clf_files if 'clf_' in f and '.pkl' in f]
    predictions = {}
    for f in clf_files:
        training_id = f.replace('clf_', '').replace('.pkl', '')
        clf = load(open('%s/%s/%s'%(results_folder, drug_folder, f), 'rb'))
        predictions[training_id] = clf.predict(X_source_response.values.astype(np.float32))
    predictions_all_drugs[drug_folder] = deepcopy(predictions)
        
    # Pred perf as pearson cor or mean squared error
    performance_GDSC = {
        training_id: [
            scipy.stats.pearsonr(p.flatten(), y_source['AUC'].values)[0],
            np.mean(np.square(p.flatten() - y_source['AUC'].values))
        ]
        for training_id, p in predictions.items()
    }

    performance_GDSC_df[drug_folder] = pd.DataFrame(performance_GDSC).T
    performance_GDSC_df[drug_folder].columns = ['corr', 'MSE']

    performance_GDSC_df[drug_folder] = performance_GDSC_df[drug_folder].merge(rank_scores_df[drug_folder],
                                                                              left_index=True, 
                                                                              right_index=True)
    
    plt.figure(figsize=(7,5))
    sns.scatterplot(data=performance_GDSC_df[drug_folder], x='MSE', y='AUC', s=75, marker='X')
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.xlabel('Mean squared error (GDSC)', fontsize=25, color='black')
    plt.ylabel('AUC (%s)'%(target_drug_name), fontsize=25, color='black')
    plt.title(GDSC_drug_name, fontsize=25, color='black')
    plt.savefig('%s/%s/GDSC_MSE_%s_AUC.png'%(results_folder, drug_folder, data_type), dpi=300)
    plt.show()
    
    plt.figure(figsize=(7,5))
    sns.scatterplot(data=performance_GDSC_df[drug_folder], x='corr', y='AUC', s=75, marker='X')
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.xlabel('Pearson correlation (GDSC)', fontsize=25, color='black')
    plt.ylabel('AUC (%s)'%(target_drug_name), fontsize=25, color='black')
    plt.title(GDSC_drug_name, fontsize=25, color='black')
    plt.savefig('%s/%s/GDSC_pearson_%s_AUC.png'%(results_folder, drug_folder, data_type), dpi=300)
    plt.show()

In [ ]:
corr_df = {
    d: df.corr()['AUC'].loc[['MSE', 'corr']]
    for d, df in performance_GDSC_df.items()
}
corr_df = pd.DataFrame(corr_df).T

In [ ]:
if data_type == 'TCGA':
    fig = (12,9)
else:
    fig = (6,9)
    
ax = corr_df.sort_values('MSE').plot.bar(y=['MSE'], figsize=fig)
ax.xaxis.set_ticklabels([e.replace('GDSC_', 'GDSC:').replace('%s_'%(data_type), '%s:'%(data_type)).replace('_', '\n')
                         for e in corr_df.sort_values('MSE').index])
plt.xticks(fontsize=15, color='black')
plt.yticks(fontsize=15, color='black')
plt.ylabel('Correlation between \n MSE (GDSC) and ROC AUC (%s)'%(data_type), fontsize=25, color='black')
plt.ylim(-1,1)
ax.get_legend().remove()
plt.tight_layout()
plt.savefig('%s/corr_GDSC_MSE_%s_AUC.png'%(figure_folder, data_type), dpi=300)

### Select best classifier on GDSC and apply it on target

In [ ]:
best_GDSC_results = {}

for drug_folder in performance_GDSC_df:
    best_mse_row = performance_GDSC_df[drug_folder].sort_values('MSE').head(1).values.astype(np.float64)
    best_GDSC_results[drug_folder] = best_mse_row.flatten()

best_GDSC_results = pd.DataFrame.from_dict(best_GDSC_results).T
best_GDSC_results.columns = performance_GDSC_df[drug_folder].columns

best_GDSC_results.to_csv('%s/%s_results_best_GDSC_MSE.csv'%(figure_folder, data_type))

### Select median classifier

In [ ]:
median_GDSC_results = {}

for drug_folder in performance_GDSC_df:
    n_median = performance_GDSC_df[drug_folder].shape[0]
    median_mse_row = performance_GDSC_df[drug_folder].sort_values('AUC', ascending=False).iloc[n_median // 2].values.astype(np.float64)
    median_GDSC_results[drug_folder] = median_mse_row.flatten()

In [ ]:
median_GDSC_results = pd.DataFrame.from_dict(median_GDSC_results).T
median_GDSC_results.columns = performance_GDSC_df[drug_folder].columns

median_GDSC_results.to_csv('%s/%s_results_median_GDSC_MSE.csv'%(figure_folder, data_type))

### Comparison of predictions between classifiers

In [ ]:
drug_key = 'GDSC_Gemcitabine_HMF_Gemcitabine'
pred_corr = [[scipy.stats.pearsonr(e.flatten(),f.flatten())[0] for e in predictions_all_drugs[drug_key].values()]
             for f in predictions_all_drugs[drug_key].values()]
pred_mse = [[np.mean(np.square(e.flatten() - f.flatten())) for e in predictions_all_drugs[drug_key].values()]
             for f in predictions_all_drugs[drug_key].values()]

pred_mse = pd.DataFrame(pred_mse,
                        columns=predictions_all_drugs[drug_key].keys(),
                        index=predictions_all_drugs[drug_key].keys())
pred_corr = pd.DataFrame(pred_corr,
                        columns=predictions_all_drugs[drug_key].keys(),
                        index=predictions_all_drugs[drug_key].keys())

In [ ]:
for e in predictions_all_drugs[drug_key].values():
    for f in predictions_all_drugs[drug_key].values():
        print(list(zip(e.flatten(), f.flatten())))
        print('\n\n\n')

In [ ]:
pred_mse = pred_mse.merge(performance_GDSC_df[drug_key],
                          left_index=True,
                          right_index=True)
pred_corr = pred_corr.merge(performance_GDSC_df[drug_key],
                          left_index=True,
                          right_index=True)

In [ ]:
for k in pred_mse.index:
    sns.distplot(pred_mse[k])
    plt.axvline(pred_mse.loc[k]['MSE'])
    plt.show()

In [ ]:
predictions_all_drugs[drug_key] = {k:f.flatten() for k,f in predictions_all_drugs[drug_key].items()}
pred_df = pd.DataFrame(predictions_all_drugs[drug_key])
for

### Random prediction

In [ ]:
random_data = np.random.normal(size=(200,data_df.shape[1]))

In [ ]:
random_predictions_all_drugs = {}

for GDSC_drug_name, GDSC_drug_ID, TCGA_drug_name in drug_list:
    print(GDSC_drug_name, TCGA_drug_name)
    X_source_response, y_source = read_GDSC_response(GDSC_drug_response_frames,
                                                     GDSC_drug_name,
                                                     data_df, GDSC_drug_ID)
    
    # Read drug gile
    drug_folder = [f for f in os.listdir(results_folder) 
                   if os.path.isdir(results_folder + f) 
                   and ('GDSC_%s'%(GDSC_drug_name) in f)
                   and ('TCGA_%s'%(TCGA_drug_name) in f)]

    if len(drug_folder) != 1:
        print('WARNING: MORE THAN ONE FOLDER')
        print(drug_folder)
        del drug_folder
    else:
        drug_folder = drug_folder[0]
        
    clf_files = os.listdir(results_folder + drug_folder)
    clf_files = [f for f in clf_files if 'clf_' in f and '.pkl' in f]
    predictions = {}
    for f in clf_files:
        training_id = f.replace('clf_', '').replace('.pkl', '')
        clf = load(open('%s/%s/%s'%(results_folder, drug_folder, f), 'rb'))
        predictions[training_id] = clf.predict(random_data.astype(np.float32))
    random_predictions_all_drugs[drug_folder] = deepcopy(predictions)

In [ ]:
drug_key = 'GDSC_Gemcitabine_TCGA_Gemcitabine'
random_predictions_all_drugs[drug_key] = {k:l.flatten() for k,l in random_predictions_all_drugs[drug_key].items()}
random_pred_corr = [[scipy.stats.pearsonr(e.flatten(),f.flatten())[0] 
                     for e in random_predictions_all_drugs[drug_key].values()]
                    for f in random_predictions_all_drugs[drug_key].values()]
random_pred_mse = [[np.mean(np.square(e.flatten() - f.flatten()))
                    for e in random_predictions_all_drugs[drug_key].values()]
                   for f in random_predictions_all_drugs[drug_key].values()]

random_predictions_df = pd.DataFrame(random_predictions_all_drugs[drug_key])

In [ ]:
sns.distplot(random_pred_mse)

In [ ]:
for e in random_predictions_df.values.T:
    sns.distplot(e)
    plt.show()

### Compare MSE to random init

In [ ]:
if baseline == 'baseline_B':
    output_cv_folder = '../deep_learning_CV/output/GDSC_only/'
elif baseline == 'baseline_C':
    output_cv_folder = '../deep_learning_CV/output/GDSC_to_%s/'%(data_type)
    
random_state = 183627362

In [ ]:
for GDSC_drug_name, GDSC_drug_ID, target_drug_name in drug_list:
    perf = []
    X_source_response, y_source = read_GDSC_response(GDSC_drug_response_frames,
                                                     GDSC_drug_name,
                                                     data_df, GDSC_drug_ID)
    drug_folder = 'GDSC_%s_%s_%s'%(GDSC_drug_name, data_type, target_drug_name)
    
    
    cv_folder = output_cv_folder + GDSC_drug_name + ('_centered_standardized' if baseline == 'baseline_B' else '')
    param = read_best_param(cv_folder, random_state)

    # Train network
    param['n_input'] = X_source_response.shape[1]
    for idx in range(20):
        print(idx)
        net = make_network(param)
        net = NeuralNetRegressor(
            net,
            max_epochs=param['n_epochs'],
            lr=param['learning_rate'],
            batch_size=param['batch_size'],
            device= 'cuda' if torch.cuda.is_available() else 'cpu',
            optimizer=torch.optim.SGD,
            optimizer__momentum=param['momentum'],
            optimizer__weight_decay=param['l2_penalty'],
            iterator_train__shuffle = True,
            verbose=0
        )
        pipeline = Pipeline([
            ('net', net)
        ])
        net.initialize()
        perf.append(np.mean(np.square(net.predict(X_source_response.values.astype(np.float32)).flatten() - y_source['AUC'])))
    break

## Processing of cross-validation results

In [ ]:
cv_results = {}
for files in os.walk(results_folder):
    relevant_files = [f for f in files[2] if 'cv_results' in f]
    if len(relevant_files) >= 1:
        cv_results[files[0].split('/')[-1]] = pd.read_csv(files[0] + '/' + relevant_files[0],
                                                         header=[0,1],
                                                          index_col=0)

In [ ]:
with pd.ExcelWriter(figure_folder + 'cv_results.xlsx') as writer:  
    for f in cv_results:
        print(f)
        cv_results[f].sort_values(('model', 'MSE')).to_excel(writer, sheet_name=f, engine='xlsxwriter')